In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [3]:
picks = pd.read_parquet('../../Data/2023_2025_pick_trades.parquet')
# Normalize franchise ownership
OWNER_REMAP = {
    "Weeks": "Gio"
}

picks = picks.copy()

picks["receiving_owner"] = picks["receiving_owner"].replace(OWNER_REMAP)
picks["giving_owner"] = picks["giving_owner"].replace(OWNER_REMAP)


In [4]:
picks[(picks['receiving_owner'] == 'Jose')].sort_values('date')

,date,receiving_owner,giving_owner,ro_picks_received,ro_picks_gave,go_picks_received,go_picks_gave,season
6,2023-09-06,Jose,Daryl,"[2024_4, 2025_1]",[],[],"[2024_4, 2025_1]",2023
16,2023-09-20,Jose,Bryan,"[2024_2, 2024_4]",[],[],"[2024_2, 2024_4]",2023
19,2023-09-22,Jose,Daryl,[],[2025_1],[2025_1],[],2023
27,2023-10-03,Jose,Luis,[2024_2],[],[],[2024_2],2023
31,2023-10-10,Jose,Luis,[2025_3],[],[],[2025_3],2023
36,2023-10-18,Jose,Gio,"[2025_2, 2025_2, 2024_2]",[],[],"[2025_2, 2025_2, 2024_2]",2023
46,2023-11-02,Jose,Brigido,"[2024_2, 2025_3]",[],[],"[2024_2, 2025_3]",2023
61,2023-11-21,Jose,Brigido,"[2024_1, 2025_2, 2026_1]",[],[],"[2024_1, 2025_2, 2026_1]",2023
62,2023-11-21,Jose,Luis,[2024_4],[],[],[2024_4],2023
65,2023-11-26,Jose,Bryan,[2025_2],"[2025_2, 2024_4]","[2025_2, 2024_4]",[2025_2],2023


In [5]:
import pandas as pd
from collections import Counter
import re

# All managers
managers = pd.unique(
    pd.concat([picks['receiving_owner'], picks['giving_owner']])
)

def starting_picks_for_manager():
    picks = []

    # Initial picks (start of 2023)
    for year in [2024, 2025, 2026]:
        for rnd in range(1, 5):
            picks.append(f"{year}_{rnd}")

    # Added at start of 2024
    for rnd in range(1, 5):
        picks.append(f"2027_{rnd}")

    # Added at start of 2025 (not tradeable, but owned)
    for rnd in range(1, 5):
        picks.append(f"2028_{rnd}")

    return Counter(picks)

# Initialize inventory
inventory = {mgr: starting_picks_for_manager() for mgr in managers}


In [6]:
VALID_PICK_PATTERN = re.compile(r"^(2024|2025|2026|2027|2028)_[1-4]$")

In [7]:
picks_sorted = picks.sort_values("date")

for _, row in picks_sorted.iterrows():
    ro = row["receiving_owner"]
    go = row["giving_owner"]

    for pick in row["ro_picks_received"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[ro][pick] += 1

    for pick in row["ro_picks_gave"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[ro][pick] -= 1

    for pick in row["go_picks_received"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[go][pick] += 1

    for pick in row["go_picks_gave"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[go][pick] -= 1


In [8]:
rows = []

for owner, picks_counter in inventory.items():
    for pick, count in picks_counter.items():
        if count > 0:
            year, rnd = pick.split("_")
            rows.append({
                "owner": owner,
                "pick_year": int(year),
                "round": int(rnd),
                "count": count
            })

pick_tracker = (
    pd.DataFrame(rows)
      .sort_values(["owner", "pick_year", "round"])
      .reset_index(drop=True)
)


### Manual Adjustments

In [25]:
# Jose: add 1 pick to 2026 Round 3
pick_tracker.loc[
    (pick_tracker["owner"] == "Jose") &
    (pick_tracker["pick_year"] == 2026) &
    (pick_tracker["round"] == 3),
    "count"
] += 1


# Bryan: remove 2 picks from 2026 Round 3
pick_tracker.loc[
    (pick_tracker["owner"] == "Bryan") &
    (pick_tracker["pick_year"] == 2026) &
    (pick_tracker["round"] == 3),
    "count"
] -= 2


# Drop any rows that now have 0 picks
pick_tracker = (
    pick_tracker
    .loc[pick_tracker["count"] > 0]
    .reset_index(drop=True)
)


In [29]:
pick_tracker_future = pick_tracker[pick_tracker['pick_year'].isin([2026, 2027, 2028])]